Import library

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import sklearn

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

#Importing keras
# from keras.utils import layer_utils
# from keras.utils.data_utils import get_file
# from keras.applications.imagenet_utils import decode_predictions
# from keras.applications.imagenet_utils import preprocess_input
# from keras.applications.imagenet_utils import _obtain_input_shape 
# from keras.engine.topology import get_source_inputs

Import dataset

In [2]:
dataset_folder = "images"

# List all files in the dataset folder
files_in_dataset = os.listdir(dataset_folder)
print(files_in_dataset)

['bagong', 'gareng', 'petruk', 'semar']


Spllit dataset

In [3]:
import splitfolders
splitfolders.ratio("images", output="output",
    seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

Data Generator

In [2]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    # rotation_range=20,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    'output/train',
    target_size=(225, 175),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

val_generator = test_datagen.flow_from_directory(
    'output/val',
    target_size=(225, 175),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    'output/test',
    target_size=(225, 175),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 2248 images belonging to 4 classes.
Found 281 images belonging to 4 classes.
Found 281 images belonging to 4 classes.


Give label to class

In [3]:
labels_dict = train_generator.class_indices
wayang_labels = dict((v, k) for k, v in labels_dict.items())
print(wayang_labels)

{0: 'bagong', 1: 'gareng', 2: 'petruk', 3: 'semar'}


In [4]:
dataset_counts = []
total_dataset_count = 0

for root, _, fileList in os.walk('output'):
    for _, label in wayang_labels.items():
        if label in root:
            dataset_counts.append(len(fileList))
            total_dataset_count += len(fileList)

print ("Total dataset: ", total_dataset_count)

Total dataset:  2810


<!-- Class Weighting -->

In [7]:
# train_classes = train_generator.num_classes
# class_labels = np.unique(train_classes)
# class_weights = sklearn.utils.class_weight.compute_class_weight('balanced', class_labels, train_classes)

# class_weights_dict = dict(zip(class_labels, class_weights))
# print("Class Weights:", class_weights)

Train model

In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(175, 225, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Replace num_classes with the number of classes in your dataset

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10

model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)


Epoch 1/10
71/71 [==============================] - 2479s 35s/step - loss: 4.2357 - accuracy: 0.2923 - val_loss: 1.4088 - val_accuracy: 0.3167
Epoch 2/10
71/71 [==============================] - 2512s 35s/step - loss: 1.3723 - accuracy: 0.3141 - val_loss: 1.3955 - val_accuracy: 0.3167
Epoch 3/10
71/71 [==============================] - 2507s 35s/step - loss: 1.3721 - accuracy: 0.3123 - val_loss: 1.3685 - val_accuracy: 0.3167
Epoch 4/10
35/71 [=============>................] - ETA: 27:14 - loss: 1.3601 - accuracy: 0.3268

Evaluate the result

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print("Test loss:", loss)
print("Test accuracy:", accuracy)